In [ ]:
from pathlib import Path
import json
import random
import numpy as np
#from CryptEnv import Environment 

In [ ]:
class Environment() :
    def __init__(self, exchange=0, path=None, dataSize=72, dataType='h', bonusData=12, defaultWalets={'EUR':100}, selectedAttrs=['close'],
                fee=0) :
        self.path = Path('data' if path==None else path)
        
        if isinstance(exchange, int) :
            exchanges = [ex for ex in self.path.glob('*') if ex.is_dir()]
            self.exchange = exchanges[exchange]
        elif isinstance(exchange, str) :
            self.exchange = self.path / exchange
            if not self.exchange.is_dir() : 
                raise ValueError("Exchange not found")

        self.dataSize = dataSize
        self.dataType = dataType
        self.bonusData = bonusData
        self.currencies = []
        self.pairs = []
        self.walets = {}
        self.defaultWalets = defaultWalets
        self.selectedAttrs = selectedAttrs
        self.load()
        self.cursor = 0
        self.reset()
        
    def load(self) :
        datas = {}
        times = []
        for file in self.exchange.glob('*.' + self.dataType) :
            self.pairs.append(file.stem)
            data = json.loads(file.read_text())
            datas[file.stem] = { int(time):d for time, d in data.items() }
            times += datas[file.stem].keys()
            
        self.pairs.sort()
        times = list(set(times))
        times.sort()
        
        self.currencies = list(set([ c for cs in self.pairs for c in cs.split('-') ]))
        self.currencies.sort()
        
        self.datas = [ {
                'time' : t,
                ** { p : 
                    ( datas[p][t] if t in datas[p] else {'close': 0, 'high': 0, 'low': 0, 'open': 0, 'volumefrom': 0, 'volumeto': 0 } )
                for p in self.pairs }
            } for i, t in enumerate(times) ]
           
            
    def getPairs(self) :
        return self.pairs
    
    def getCurrencies(self) :
        return self.currencies
    
    def reset(self) :
        self.cursor = random.randrange(self.dataSize-1, len(self.datas) - self.bonusData)
        self.walets = { c:( self.defaultWalets[c] if c in self.defaultWalets else 0 ) for c in self.currencies }
    
    def readData(self) :
        self.cursor += 1
        if self.cursor + self.bonusData >= len(self.datas) : 
            return None
        
        walets = self.walets.values()
        past = [ [ self.datas[t][p][a] for p in self.pairs for a in self.selectedAttrs ] for t in range(self.cursor - self.dataSize,self.cursor) ]
        futur = [ [ self.datas[t][p][a] for p in self.pairs for a in self.selectedAttrs ] for t in range(self.cursor+1, self.cursor+self.bonusData+1) ]
        
        return walets, past, futur
    
    def getShapes(self) :
        input = (
            len(self.walets), 
            [self.dataSize, len(self.pairs)*len(self.selectedAttrs) ], 
            [self.bonusData, len(self.pairs)*len(self.selectedAttrs) ]
        )
        output = len(self.walets)-1
        
        return input, output
    
    def apply(self, actions) :
        i = 0
        for c in self.currencies : 
            if c!='EUR' :
                p = c + '-EUR'
                r = self.datas[self.cursor][p]['close']
                if r>0 :
                    self.walets['EUR'] -= actions[i]
                    self.walets[c] += actions[i] / r
                i+=1

    def eval(self) :
        ret = 0
        for c, n in self.walets.items() : 
            if n<0 : return 0
            
            if c=='EUR' :
                r = 1.1
            else :
                p = c + '-EUR'
                r = self.datas[self.cursor][p]['close']
            ret += self.walets[c] * r
        
        return ret


In [ ]:
#import CryptEnv

#c = Environment(exchange='Kraken', selectedAttrs=['close','open'])
c = Environment(exchange='Kraken')

In [ ]:
#print(c.path, c.exchange, c.datas['ADA-EUR'])
#print(len(c.datas), c.datas[49916])
random.seed(42)
#c.reset()
c.cursor = 49800
print(c.getShapes())
print(c.eval())
print(c.walets)
print(c.apply([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1]))
print(c.walets)
print(c.eval())

#walets, past, futur = c.readData()
#print(walets)
#print('---------------')
#print(past)
#print('---------------')
#print(futur)
#print(c.cursor, c.dataSize, c.bonusData, len(c.datas), c.readData())
#print(c.pairs, c.currencies)